# preprocessing

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv("0714train.csv")
df = df.drop(columns='Number')
REF_df = df.dropna()
print('has {} null in all data\n\n'.format(df.isnull().sum().sum()))
print(df.dtypes.value_counts())


In [ ]:
cat = df.select_dtypes(include='object').columns
num = df.select_dtypes(include=['float64','int64']).columns

# Fill NA   (cat and num)

####  RNN

In [ ]:
from sklearn.neighbors import RadiusNeighborsRegressor,KNeighborsRegressor
dis = 3
for i in range(df.shape[0]):
    Na_List = set(df.columns[df.iloc[i].isna() ].tolist())
    Na_List-=set(cat)
    for j in list(Na_List):
        i2 = df.columns.get_loc(j)
        if '_C_' in j:
            neigh = RadiusNeighborsRegressor(radius=dis,weights='distance')
            neigh.fit(REF_df.iloc[:,-6:],REF_df.iloc[:,i2])
            df.iloc[i,i2]=neigh.predict(np.array(df.iloc[i,-6:]).reshape(1, -1))
        elif '_A1_' in j:
            neigh = RadiusNeighborsRegressor(radius=dis,weights='distance')
            neigh.fit(REF_df.iloc[:,-6].reshape(-1, 1),REF_df.iloc[:,i2])
            df.iloc[i,i2]=neigh.predict(df.iloc[i,-6].reshape(-1, 1))
        elif '_A2_' in j:
            neigh = RadiusNeighborsRegressor(radius=dis,weights='distance')
            neigh.fit(REF_df.iloc[:,-5].reshape(-1, 1),REF_df.iloc[:,i2])
            df.iloc[i,i2]=neigh.predict(df.iloc[i,-5].reshape(-1, 1))
        elif '_A3_' in j:
            neigh = RadiusNeighborsRegressor(radius=dis,weights='distance')
            neigh.fit(REF_df.iloc[:,-4].reshape(-1, 1),REF_df.iloc[:,i2])
            df.iloc[i,i2]=neigh.predict(df.iloc[i,-4].reshape(-1, 1))
        elif '_A4_' in j:
            neigh = RadiusNeighborsRegressor(radius=dis,weights='distance')
            neigh.fit(REF_df.iloc[:,-3].reshape(-1, 1),REF_df.iloc[:,i2])
            df.iloc[i,i2]=neigh.predict(df.iloc[i,-3].reshape(-1, 1))
        elif '_A5_' in j:
            neigh = RadiusNeighborsRegressor(radius=dis,weights='distance')
            neigh.fit(REF_df.iloc[:,-2].reshape(-1, 1),REF_df.iloc[:,i2])
            df.iloc[i,i2]=neigh.predict(df.iloc[i,-2].reshape(-1, 1))
        elif '_A6_' in j:
            neigh = RadiusNeighborsRegressor(radius=dis,weights='distance')
            neigh.fit(REF_df.iloc[:,-1].reshape(-1, 1),REF_df.iloc[:,i2])
            df.iloc[i,i2]=neigh.predict(df.iloc[i,-1].reshape(-1, 1))

In [ ]:
print('has {} null in all data\n\n'.format(df.isnull().sum().sum()))

# Fill category

In [ ]:
def Convert(s):
    s=str(s)
    L=s.split(';')
    out=''
    if len(L)==1:
#         print(i)
        return 'EMPTY'
    if L[0]=='U':
        out+=str(L[1])
    elif L[0]=='D':
        out+='-'+str(L[1])
    else:
        out+='0'
    out+=';'
    if L[2]=='R':
        out+=str(L[3])
    elif L[2]=='L':
        out+='-'+str(L[3])
    else:
        out+='0'
    return out

In [ ]:
# make REF
cat2=[]
for i in cat:
    REF_df[i]=REF_df[i].apply(Convert)
    REF_df['%s_x'%i]=REF_df['%s'%i].apply(lambda x: np.nan if pd.isna(x) else pd.Series(x.split(';')[0])).astype('float')
    REF_df['%s_y'%i]=REF_df['%s'%i].apply(lambda x: np.nan if pd.isna(x) else pd.Series(x.split(';')[1])).astype('float')
    cat2.append('%s_x'%i)
    cat2.append('%s_y'%i)
    REF_df=REF_df.drop(columns=[i])
REF_df = REF_df.reindex_axis(sorted(REF_df.columns), axis=1)

In [ ]:
cat2=[]
for i in cat:
    df[i]=df[i].apply(Convert)
    df['%s_x'%i]=df['%s'%i].apply(lambda x: REF_df['%s_x'%i].mean() if 'EMPTY' in x else (x.split(';')[0])).astype('float')
    df['%s_y'%i]=df['%s'%i].apply(lambda x: REF_df['%s_y'%i].mean() if 'EMPTY' in x else (x.split(';')[1])).astype('float')
    
    cat2.append('%s_x'%i)
    cat2.append('%s_y'%i)
    
    df=df.drop(columns=[i])
df = df.reindex_axis(sorted(df.columns), axis=1)

In [ ]:
from sklearn.neighbors import RadiusNeighborsRegressor,KNeighborsRegressor
dis = 3
for i in range(df.shape[0]):
    Na_List = set(df.columns[df.iloc[i].isna() ].tolist())
    for j in list(Na_List):
        i2 = df.columns.get_loc(j)
        neigh = RadiusNeighborsRegressor(radius=dis,weights='distance')
        neigh.fit(REF_df.iloc[:,-6:],REF_df.iloc[:,i2])
        df.iloc[i,i2]=neigh.predict(np.array(df.iloc[i,-6:]).reshape(1, -1))

In [ ]:
print('has {} null in all data\n\n'.format(df.isnull().sum().sum()))


In [ ]:
df.to_csv("pre-0714train.csv",index=False)